<a href="https://colab.research.google.com/github/stratis-forge/registration-workflows/blob/main/demo_deformable_registration_ANTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Deformable registration using ANTs

## Requirements

* [ANTs](https://github.com/ANTsX/ANTs) for deformable image registration
* [Plastimatch](https://plastimatch.org/) for data conversion

## Convert input DICOMs to NIfTI

In [ ]:
import os, glob
import SimpleITK as sitk
import numpy as np
from rt_utils import RTStructBuilder

In [1]:
#I/O paths

#Input DICOM data
dcm_path = '/lab/deasylab1/Aditi/forWei/sample_dataset/Dcm/Pt2';

planning_ct_path = os.path.join(dcm_path,'CT_20130509_UNKNOWN');
pet_ct_path = os.path.join(dcm_path,'CT_20130509_CT_HN');
warped_scan_dcm = os.path.join(dcm_path, 'Warped','Scan')
warped_mask_dcm = os.path.join(dcm_path, 'Warped','Mask')
os.mkdir(warped_scan_dcm)
os.mkdir(warped_mask_dcm)


#Input NIfTI data
nii_path = '/lab/deasylab1/Aditi/forWei/sample_dataset/Nii_Pt2'
os.makedirs(nii_path, exist_ok = True)

moving_img_nii = os.path.join(nii_path, 'ctmoving.nii')
fixed_img_nii = os.path.join(nii_path, 'ctfixed.nii')

Convert scans

In [2]:
%%capture
%%bash -s {pet_ct_path} {fixed_img_nii}
plastimatch convert --input $1 \
--output-img $2

In [3]:
%%capture
%%bash -s {planning_ct_path} {moving_img_nii}
plastimatch convert --input $1 \
--output-img $2

Convert mask 

In [4]:
moving_mask_dcm = r"/path/to/rtstruct.dcm"
# moving_mask_nii = os.path.join(nii_path,'strMask.nii')
#str_list = os.path.join(nii_path,'strList.txt')

In [5]:
# %%capture
# %%bash -s {moving_mask_dcm} {moving_mask_nii} {str_list} {planning_ct_path} 
# plastimatch convert --input $1 \
# --output-ss-img $2 \
# --output-ss-list $3 \
# --referenced-ct $4

## Register images using ANTs

In [6]:
transform_type = 's'

#prefix of final coregistred files
out_nii_prefix = os.path.join(nii_path,'Moving_to_fixed')

In [7]:
transform1 = out_nii_prefix + '1Warp.nii.gz'
transform2 = out_nii_prefix + '0GenericAffine.mat'
transform2Txt = out_nii_prefix + '0GenericAffine.txt'
composed_transform_ants = os.path.join(nii_path, 'Moving_to_fixedComposedAnts.nii')

warped_scan = out_nii_prefix + 'warpedScan.nii.gz'
warped_mask = out_nii_prefix + 'warpedMask.nii.gz'

In [8]:
%%capture
%%bash -s {fixed_img_nii} {moving_img_nii} {transform_type} {out_nii_prefix}

# Compute deformation
antsRegistrationSyNQuick.sh -d 3 -f $1 -m $2 -t $3 -o $4 -n 4

In [9]:
%%bash -s {fixed_img_nii} {transform1} {transform2} {composed_transform_ants}
ComposeMultiTransform 3 $4 -R $1 $2 $3

## Warp moving image and structures using Plastimatch

Warp scan

In [10]:
%%capture
%%bash -s {planning_ct_path} {warped_scan_dcm} {composed_transform_ants}
plastimatch warp \
  --input $1 \
  --output-dicom $2 \
  --xf $3

Warp GTV mask

In [11]:
%%capture
%%bash -s {moving_mask_dcm} {warped_mask_dcm} {composed_transform} {warped_scan_dcm}
plastimatch warp \
  --input $1 \
  --output-dicom $2 \
  --xf $3 \
  --referenced-ct $4

## Import to CERR

In [ ]:
# %load_ext oct2py.ipython

In [ ]:
# %octave_push warped_scan_out warped_mask_out

In [ ]:
# %%octave
# cerr_path = '/cluster/home/stratis/CERR/';
# curr_dir = pwd;
# cd(cerr_path);
# addToPath2(cerr_path);
# cd(curr_dir);

# #Load original data
# orig_data = '/lab/deasylab1/Aditi/forWei/sample_dataset/CERR/Pt1.mat';
# out_file = '/lab/deasylab1/Aditi/forWei/sample_dataset/CERR/Pt1_out.mat';
# planC = loadPlanC(orig_data,tempdir);
# planC = quality_assure_planC(orig_data,planC);
# planC = updatePlanFields(planC);
# indexS = planC{end};

In [ ]:
# %%octave
# #Import warped planning CT
# scanName = 'Warped_planning_CT';
# planC = nii2cerr(warped_scan_out,scanName,planC);

# #Import warped mask
# [mask3M,planC] = readNiftiStructureSet(ssFilename,ssList,...
# {'GTVp','GTVn','GTVn_indetermina'},4,planC);

# save_planC(planC,[],'passed',out_file);